Part #1 Segmenting and Clustering Assignment

*Data Gathering*

In [3]:
pip install bs4

     |████████████████████████████████| 102kB 18.7MB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install lxml

     |████████████████████████████████| 5.8MB 25.5MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# import necessary libraries for data analysis and processing.
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib3.request
from sklearn.cluster import KMeans
import folium
import os
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
# get data from wiki and create soup object.
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

In [9]:
#use our soup object to extract wikitable data from html page and store it into a list.
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #set first row of the list to be its header.
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into pd DataFrame.
ca_df = pd.DataFrame(data = data,columns = columns)
ca_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


*Data Wrangling*

In [11]:
#new dataframe with Neighbourhoods that are labeled as not assigned.
dfN=ca_df
ca_df1 = dfN.loc[dfN['Neighbourhood']== 'Not assigned']

#replace Not assigned Neighbourhoods column values with Borough column.
for i in ca_df1.index:
    ca_df1.at[i, 'Neighbourhood'] = ca_df1.at[i, 'Borough']

#replace rows in DataFrame.
dfN.loc[ca_df1.index] = ca_df1

#delete rows with Borough that are not assigned.
ca_df = dfN[dfN.Borough != 'Not assigned']

#group by postcode and Borough, keeping all the Neighbourhoods.
ca_df=ca_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()

#remove duplicates.
ca_df = ca_df.drop_duplicates()
ca_df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [12]:
ca_df.shape

(103, 3)